## Lab 3

### Part 1. Overfit it (1.5 points)

Будем работать с датасетом [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) (*hint: он доступен в torchvision*).

Ваша задача состоит в следующем:
1. Обучить сеть, которая покажет >= 0.92 test accuracy.
2. Пронаблюдать и продемонстрировать процесс переобучения сети с увеличением числа параметров (==нейронов) и/или числа слоев и продемонстрировать это наглядно (например, на графиках).
3. Попробовать частично справиться с переобучением с помощью подходящих приемов (Dropout/batchnorm/augmentation etc.)

*Примечание*: Пункты 2 и 3 взаимосвязаны, в п.3 Вам прелагается сделать полученную в п.2 сеть менее склонной к переобучению. Пункт 1 является независимым от пунктов 2 и 3.

# Preparations

### Imports

In [1]:
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np

### Work with data

In [2]:
batch_size = 128
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.], [0.5])])

In [4]:
train_data = torchvision.datasets.FashionMNIST(root='data/', train=True, download=True, transform=transform)
test_data = torchvision.datasets.FashionMNIST(root='data/', train=False, download=True, transform=transform)

In [5]:
print(len(train_data))
print(len(test_data))

60000
10000


In [6]:
print(f'images shape = {train_data.data.shape}')

images shape = torch.Size([60000, 28, 28])


In [7]:
num_classes = len(train_data.classes)
print(f'number of classes = {len(train_data.classes)}')

number of classes = 10


In [8]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Trainer

In [9]:
def train(model, optimizer, criterion, train, test, n_epochs=10, device='cpu'):
    model = model.to(device)
    
    for ep in range(n_epochs):
        model.train()
        losses = []
        i = 0
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            output = model(x)
        
            loss = criterion(output, y)
            losses.append(float(loss.cpu()))
            loss.backward()
            optimizer.step()
            i += 1
            if not i % 15:
                print(i)
            
        print(f'epoch={ep}, loss={np.mean(losses)}')
    
    return net

# Problem 1

### Building Neural Network

In [10]:
class CustomFlatten(nn.Module):
    
    def __init__(self, size):
        super(CustomFlatten, self).__init__()
        self.size = size
        
    def forward(self, x):
        return x.view(-1, self.size)

    
class BasicNet(nn.Module):
    
    def __init__(self):
        super(BasicNet, self).__init__()
        self.basic_net = nn.Sequential(
                # shape: (1, 28, 28)
                nn.Conv2d(in_channels=1, out_channels=10, kernel_size=2, stride=2), 
                nn.ReLU(),
                # shape: (10, 14, 14)
                nn.Conv2d(in_channels=10, out_channels=25, kernel_size=2, stride=2),
                nn.ReLU(),
                # shape: (25, 7, 7)
                CustomFlatten(25*7*7),
                nn.Linear(in_features=25*7*7, out_features=150),
                nn.ReLU(),
                nn.Linear(in_features=150, out_features=10),
            )
        
    def forward(self, x):
        return self.basic_net(x)

In [11]:
net = BasicNet()
optimizer = torch.optim.Adam(params=net.parameters())
criterion = nn.CrossEntropyLoss()
net = train(net, optimizer, criterion, train_loader, test_loader)

15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=0, loss=0.5645562957789598
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=1, loss=0.3668308152255219
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=2, loss=0.3212789487419352
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=3, loss=0.28913878190364917
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=4, loss=0.26867127418518066
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
epoch=5, loss=0.2488617638407994
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
37